In [4]:
import os
os.chdir('C:\\Users\\kaush\\OneDrive\\Desktop\\UT\\CS 363M\\ML Project')
print(os.getcwd())

C:\Users\kaush\OneDrive\Desktop\UT\CS 363M\ML Project


In [50]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from catboost import CatBoostRegressor
import xgboost as xgb



In [51]:
# Load data
train = pd.read_csv("cattle_data_train.csv")
test = pd.read_csv("cattle_data_test.csv")

In [ ]:
for i in train.columns:
    # print(train[i].dtype)
    print(train[i].name, ":", train[i].dtype)


In [52]:
# Target + ID
target = "Milk_Yield_L"
id_col = "Cattle_ID"

# Basic preprocessing
# (turn categories -> strings -> numeric encodings)
def preprocess(df):
    milk_test = df.copy()
    milk_test["Date"] = pd.to_datetime(milk_test["Date"])
    milk_test['year'] = milk_test['Date'].dt.year
    milk_test['month'] = milk_test['Date'].dt.month
    milk_test['day'] = milk_test['Date'].dt.day
    milk_test['dayofweek'] = milk_test['Date'].dt.dayofweek
    milk_test['weekofyear'] = milk_test['Date'].dt.isocalendar().week.astype(int)
    milk_test['quarter'] = milk_test['Date'].dt.quarter
    milk_test['is_weekend'] = milk_test['dayofweek'].isin([5, 6]).astype(int)
    milk_test['date_ordinal'] = milk_test['Date'].map(pd.Timestamp.toordinal)
    milk_test['Breed'] = milk_test['Breed'].str.strip()
    milk_test['Breed'] = milk_test['Breed'].replace({'Holstien': 'Holstein'})
    milk_test['Feed_Quantity_kg'] = milk_test.groupby('Feed_Type')['Feed_Quantity_kg'].transform(lambda x: x.fillna(x.median()))
    milk_test['Housing_Score'] = milk_test['Housing_Score'].transform(lambda x: x.fillna(x.median()))
    milk_test = milk_test.drop("Feed_Quantity_lb", axis=1)
    milk_test = milk_test.drop(id_col, axis=1)
    milk_test = milk_test.drop("Date", axis=1)
    for col in milk_test.columns:
        if milk_test[col].dtype == "object":
            milk_test[col] = milk_test[col].astype("category").cat.codes
    return milk_test

train_prep = preprocess(train.drop(columns=[target]))
test_prep = preprocess(test)

y = train[target]
X = train_prep

In [40]:
# 5-Fold Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

fold_rmse = []
models = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    print(f"\n----- Fold {fold+1} -----")

    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    model = lgb.LGBMRegressor(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=-1,
        num_leaves=64,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    )

    model.fit(X_train, y_train)

    preds = model.predict(X_val)
    mse = mean_squared_error(y_val, preds)
    rmse = np.sqrt(mse)
    fold_rmse.append(rmse)

    print(f"Fold {fold+1} RMSE: {rmse:.4f}")
    models.append(model)

print("\n==========================")
print(f"Average RMSE: {np.mean(fold_rmse):.4f}")
print("==========================\n")


----- Fold 1 -----
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.013554 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3623
[LightGBM] [Info] Number of data points in the train set: 168000, number of used features: 40
[LightGBM] [Info] Start training from score 15.587505
Fold 1 RMSE: 4.1447

----- Fold 2 -----
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.017294 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3622
[LightGBM] [Info] Number of data points in the train set: 168000, number of used features: 40
[LightGBM] [Info] Start training from score 15.580781
Fold 2 RMSE: 4.1141

----- Fold 3 -----
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.014441 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3623
[LightGBM] [Info

In [46]:
# 5-Fold Cross-Validation for CatBoost
kf = KFold(n_splits=5, shuffle=True, random_state=42)

cat_fold_rmse = []
cat_models = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    print(f"\n----- CatBoost Fold {fold+1} -----")

    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    model = CatBoostRegressor(
        loss_function="RMSE",
        n_estimators=1000,
        learning_rate=0.05,
        depth=8,
        subsample=0.8,
        random_seed=42,
        verbose=False  # turn off big CatBoost logs
    )

    model.fit(X_train, y_train, eval_set=(X_val, y_val), verbose=False)

    preds = model.predict(X_val)
    mse = mean_squared_error(y_val, preds)
    rmse = np.sqrt(mse)
    cat_fold_rmse.append(rmse)

    print(f"CatBoost Fold {fold+1} RMSE: {rmse:.4f}")
    cat_models.append(model)

print("\n==========================")
print(f"CatBoost Average RMSE: {np.mean(cat_fold_rmse):.4f}")
print("==========================\n")



----- CatBoost Fold 1 -----
CatBoost Fold 1 RMSE: 4.1297

----- CatBoost Fold 2 -----
CatBoost Fold 2 RMSE: 4.0945

----- CatBoost Fold 3 -----
CatBoost Fold 3 RMSE: 4.0998

----- CatBoost Fold 4 -----
CatBoost Fold 4 RMSE: 4.1249

----- CatBoost Fold 5 -----
CatBoost Fold 5 RMSE: 4.1191

CatBoost Average RMSE: 4.1136



In [53]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

fold_rmse = []
xgb_models = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    print(f"\n----- XGBoost Fold {fold+1} -----")
    
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    
    model = xgb.XGBRegressor(
        n_estimators=700,
        learning_rate=0.05,
        max_depth=8,
        subsample=0.8,
        colsample_bytree=0.8,
        objective="reg:squarederror",
        tree_method="hist",      # fast for large data
        random_state=42
    )
    
    model.fit(X_train, y_train)
    
    preds = model.predict(X_val)
    mse = mean_squared_error(y_val, preds)
    rmse = np.sqrt(mse)
    fold_rmse.append(rmse)
    
    print(f"Fold {fold+1} RMSE: {rmse:.4f}")
    xgb_models.append(model)

print("\n==========================")
print(f"XGBoost Average RMSE = {np.mean(fold_rmse):.4f}")
print("==========================\n")


----- XGBoost Fold 1 -----
Fold 1 RMSE: 4.1783

----- XGBoost Fold 2 -----
Fold 2 RMSE: 4.1537

----- XGBoost Fold 3 -----
Fold 3 RMSE: 4.1538

----- XGBoost Fold 4 -----
Fold 4 RMSE: 4.1768

----- XGBoost Fold 5 -----
Fold 5 RMSE: 4.1725

XGBoost Average RMSE = 4.1670



In [54]:
# Train final CatBoost model on full data
final_cat_model = CatBoostRegressor(
    loss_function="RMSE",
    n_estimators=1000,
    learning_rate=0.05,
    depth=8,
    subsample=0.8,
    random_seed=42,
    verbose=False
)
final_cat_model.fit(X, y, verbose=False)

# Make submission
cat_test_preds = final_cat_model.predict(test_prep)

cat_submission = pd.DataFrame({
    id_col: test[id_col],
    target: cat_test_preds
})

cat_submission.to_csv("submission_catboost.csv", index=False)
print("submission_catboost.csv created!")

submission_catboost.csv created!


In [9]:
# -----------------------
# Train final model on full data
# -----------------------
final_model = lgb.LGBMRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=-1,
    num_leaves=64,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
final_model.fit(X, y)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.011718 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3987
[LightGBM] [Info] Number of data points in the train set: 210000, number of used features: 35
[LightGBM] [Info] Start training from score 15.589156


LGBMRegressor(colsample_bytree=0.8, learning_rate=0.05, n_estimators=500,
              num_leaves=64, random_state=42, subsample=0.8)

In [10]:
# -----------------------
# Create submission
# -----------------------
test_preds = final_model.predict(test_prep)

submission = pd.DataFrame({
    id_col: test[id_col],
    target: test_preds
})

submission.to_csv("submission.csv", index=False)
print("submission.csv created!")

submission.csv created!
